In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import yaml
from utils import *
import sys

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
config_filename = "D:/Projects/SANDAG/client_sandag_off_model_calculators/advanced_traffic_management_systems/data/config.yml"

In [4]:
if not os.path.exists(config_filename):
    msg = "Configuration file doesn't exist at: {}".format(config_filename)
    raise ValueError(msg)

with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
# inputs
highway_load_am_file = config['inputs']['highway_load_am_file']
highway_load_pm_file = config['inputs']['highway_load_pm_file']
highway_load_md_file = config['inputs']['highway_load_md_file']
highway_load_ea_file = config['inputs']['highway_load_ea_file']
highway_load_ev_file = config['inputs']['highway_load_ev_file']

highway_network_shapefile = config['inputs']['highway_network_shapefile']

atms_strategy_file = config['inputs']['atms_strategy_file']
emission_factors_file = config['inputs']['emission_factors_file']
intersection_delays_file = config['inputs']['intersection_delays_file']

In [6]:
# outputs
output_dir = config['outputs']['output_dir']
output_results_filename = config['outputs']['output_file_name']

In [7]:
# parameters
scen_year = config['parameters']['scen_year']
percent_atms_delay_reduction = config['parameters']['percent_atms_delay_reduction']
apply_adaptive_signal_reduction = config['parameters']['apply_adaptive_signal_reduction']

In [8]:
if apply_adaptive_signal_reduction:
    percent_adaptive_signal_reduction = config['parameters']['percent_adaptive_signal_reduction']
    total_percent_delay_reduction = percent_atms_delay_reduction + percent_adaptive_signal_reduction
else: 
    total_percent_delay_reduction = percent_atms_delay_reduction

In [9]:
# other paramteters
MAX_DELAY = 0.8

In [10]:
with open(atms_strategy_file, "r") as yml_file:
    atms_config = yaml.safe_load(yml_file)
    atms_strategy_dict = atms_config['intersections']
    
number_of_atms_intersections = len(atms_strategy_dict["intersection_node_id"])

In [11]:
# read inputs 
links_df = gpd.read_file(highway_network_shapefile)

highway_load_am_df = pd.read_csv(highway_load_am_file)
highway_load_pm_df = pd.read_csv(highway_load_pm_file)
highway_load_md_df = pd.read_csv(highway_load_md_file)
highway_load_ea_df = pd.read_csv(highway_load_ea_file)
highway_load_ev_df = pd.read_csv(highway_load_ev_file)

emission_df = pd.read_excel(emission_factors_file)

In [12]:
# intersection delays
if (atms_strategy_dict["intersection_delays_in_minutes"]) == None:
    USER_SPECIFIED_DELAY = 0
    
    intersection_delays_df = pd.read_csv(intersection_delays_file)
    intersection_delays_df = intersection_delays_df[["From", "To", "Inter_Time_EA",  "Inter_Time_AM",  "Inter_Time_MD", "Inter_Time_PM", "Inter_Time_EV"]]
    intersection_delays_df = intersection_delays_df.drop_duplicates()
    intersection_delays_df.rename(
        columns={
            "Inter_Time_EA": "ea_delay",
            "Inter_Time_AM": "am_delay",        
            "Inter_Time_MD": "md_delay",
            "Inter_Time_PM": "pm_delay",
            "Inter_Time_EV": "ev_delay"
         },
        inplace=True
    )
else: 
    USER_SPECIFIED_DELAY = 1
    
    intersection_delays_df = pd.DataFrame({'node_id': atms_strategy_dict["intersection_node_id"], 'delay': atms_strategy_dict["intersection_delays_in_minutes"]})
    
    if max(intersection_delays_df["delay"]) > MAX_DELAY:
        sys.exit('The user specifid delay is more than the maximum allowed delay for one or more intersections')
        
    cols = ['ea_delay','am_delay','md_delay','pm_delay', 'ev_delay']
    
    for c in cols:
        intersection_delays_df[c] = intersection_delays_df['delay']
        
    intersection_delays_df = intersection_delays_df.drop(columns = ["delay"])

In [13]:
highway_load_data_dict = {"AM": highway_load_am_df, "PM": highway_load_pm_df, "MD": highway_load_md_df, "EA": highway_load_ea_df, "EV": highway_load_ev_df}

In [14]:
# get two approaching links for each atms intersection
corridor_links_df = get_corridor_links(links_df, atms_strategy_dict)
intersection_links_df = corridor_links_df[corridor_links_df.intersection > 0]

In [15]:
# get link attributes for approaching links for all intersections
intersection_links_df = get_link_attributes(intersection_links_df, highway_load_data_dict)

In [16]:
data_df = intersection_links_df.copy()

data_df["am_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_am_flow"], data_df["ba_am_flow"])
data_df["pm_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_flow"], data_df["ba_pm_flow"])
data_df["md_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_md_flow"], data_df["ba_md_flow"])
data_df["ea_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_ea_flow"], data_df["ba_ea_flow"])
data_df["ev_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_ev_flow"], data_df["ba_ev_flow"])

data_df["am_auto_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_am_auto_flow"], data_df["ba_am_auto_flow"])
data_df["pm_auto_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_auto_flow"], data_df["ba_pm_auto_flow"])
data_df["md_auto_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_md_auto_flow"], data_df["ba_md_auto_flow"])
data_df["ea_auto_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_ea_auto_flow"], data_df["ba_ea_auto_flow"])
data_df["ev_auto_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_ev_auto_flow"], data_df["ba_ev_auto_flow"])

data_df["am_truck_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_am_truck_flow"], data_df["ba_am_truck_flow"])
data_df["pm_truck_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_pm_truck_flow"], data_df["ba_pm_truck_flow"])
data_df["md_truck_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_md_truck_flow"], data_df["ba_md_truck_flow"])
data_df["ea_truck_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_ea_truck_flow"], data_df["ba_ea_truck_flow"])
data_df["ev_truck_flow"] = np.where(data_df["approach"] == "AB", data_df["ab_ev_truck_flow"], data_df["ba_ev_truck_flow"])

data_df["from_node"] = np.where(data_df["approach"] == "AB", data_df["from_node"], data_df["to_node"])
data_df["to_node"] = np.where(data_df["approach"] == "AB", data_df["to_node"], data_df["from_node"])

In [17]:
# join intersection link flows and delays into one dataframe
flows_df = data_df.copy()

if USER_SPECIFIED_DELAY == 0:
    flows_df = pd.merge(
        flows_df,
        intersection_delays_df,
        how="left",
        left_on=["from_node", "to_node"],
        right_on=["From", "To"]
    )
else:   
    flows_df = pd.merge(
        flows_df,
        intersection_delays_df,
        how="left",
        left_on=["to_node"],
        right_on=["node_id"]
    )

In [18]:
flows_df

,hwycov_id,link_name,from_node,to_node,corridor,intersection,approach,ab_am_flow,ba_am_flow,ab_am_auto_flow,ba_am_auto_flow,ab_am_truck_flow,ba_am_truck_flow,ab_pm_flow,ba_pm_flow,ab_pm_auto_flow,ba_pm_auto_flow,ab_pm_truck_flow,ba_pm_truck_flow,ab_md_flow,ba_md_flow,ab_md_auto_flow,ba_md_auto_flow,ab_md_truck_flow,ba_md_truck_flow,ab_ea_flow,ba_ea_flow,ab_ea_auto_flow,ba_ea_auto_flow,ab_ea_truck_flow,ba_ea_truck_flow,ab_ev_flow,ba_ev_flow,ab_ev_auto_flow,ba_ev_auto_flow,ab_ev_truck_flow,ba_ev_truck_flow,am_flow,pm_flow,md_flow,ea_flow,ev_flow,am_auto_flow,pm_auto_flow,md_auto_flow,ea_auto_flow,ev_auto_flow,am_truck_flow,pm_truck_flow,md_truck_flow,ea_truck_flow,ev_truck_flow,From,To,ea_delay,am_delay,md_delay,pm_delay,ev_delay
0,6557,EL CAJON,15380,15381,Primary,1.0,AB,1979.406860,2690.896973,1958.778651,2650.571520,20.628209,40.325453,3318.066162,3519.246826,3285.662698,3478.497947,32.403464,40.748879,5232.598145,5649.671875,5098.271929,5517.170161,134.326216,132.501714,282.750916,382.102661,280.424816,379.528955,2.326100,2.573706,1946.161987,1771.498291,1927.899945,1760.518818,18.262042,10.979473,1979.406860,3318.066162,5232.598145,282.750916,1946.161987,1958.778651,3285.662698,5098.271929,280.424816,1927.899945,20.628209,32.403464,134.326216,2.326100,18.262042,15380.0,15381.0,0.17,0.17,0.17,0.17,0.17
1,6558,EL CAJON,15382,15382,Primary,1.0,BA,1827.766479,2306.507080,1811.078249,2277.974003,16.688230,28.533077,3357.654541,2556.724854,3331.692010,2520.939386,25.962531,35.785468,5327.767090,4337.027344,5192.873139,4235.419127,134.893951,101.608217,255.229462,353.400238,252.785702,351.906533,2.443760,1.493705,2233.034424,1286.431641,2217.609078,1278.833670,15.425346,7.597971,2306.507080,2556.724854,4337.027344,353.400238,1286.431641,2277.974003,2520.939386,4235.419127,351.906533,1278.833670,28.533077,35.785468,101.608217,1.493705,7.597971,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6559,EL CAJON,30914,15385,Primary,2.0,AB,2144.520264,3274.759766,2128.074322,3223.832262,16.445942,50.927504,3238.226074,4158.672852,3203.274465,4104.712459,34.951609,53.960393,5602.447754,6583.221191,5443.108869,6419.014032,159.338885,164.207159,377.867676,453.202484,376.106686,448.986946,1.760990,4.215538,2165.466797,1992.258545,2140.354867,1968.254532,25.111930,24.004013,2144.520264,3238.226074,5602.447754,377.867676,2165.466797,2128.074322,3203.274465,5443.108869,376.106686,2140.354867,16.445942,34.951609,159.338885,1.760990,25.111930,30914.0,15385.0,0.17,0.17,0.17,0.17,0.17
3,3659,EL CAJON,15386,15386,Primary,2.0,BA,2445.473877,2717.082275,2424.785880,2671.479871,20.687997,45.602404,3407.487305,3374.456787,3369.739675,3330.462647,37.747630,43.994140,6162.441895,5198.140137,5970.799033,5058.914403,191.642862,139.225734,444.174713,370.091553,442.392703,366.916015,1.782010,3.175538,2140.299072,1523.906616,2110.927763,1500.422603,29.371309,23.484013,2717.082275,3374.456787,5198.140137,370.091553,1523.906616,2671.479871,3330.462647,5058.914403,366.916015,1500.422603,45.602404,43.994140,139.225734,3.175538,23.484013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3660,EL CAJON,15386,15387,Primary,3.0,AB,2445.473877,2717.082275,2424.785880,2671.479871,20.687997,45.602404,3407.487305,3374.456787,3369.739675,3330.462647,37.747630,43.994140,6162.441895,5198.140137,5970.799033,5058.914403,191.642862,139.225734,444.174713,370.091553,442.392703,366.916015,1.782010,3.175538,2140.299072,1523.906616,2110.927763,1500.422603,29.371309,23.484013,2445.473877,3407.487305,6162.441895,444.174713,2140.299072,2424.785880,3369.739675,5970.799033,442.392703,2110.927763,20.687997,37.747630,191.642862,1.782010,29.371309,15386.0,15387.0,0.17,0.17,0.17,0.17,0.17
5,5236,EL CAJON,15389,15389,Primary,3.0,BA,3707.487061,3172.351074,3659.556977,3125.268669,47.930084,47.082405,4863.750977,3968.783691,4811.377258,3924.116619,52.373719,44.667072,9148.836914,6133.930176,8835.515779,5984.797133,313.321135,149.133043,551.431335,358.091553,547.794654,354.916015,3.636681,3.175538,2855.872314,2072.539551,2813.829539,203

In [19]:
total_percent_delay_reduction

12.0

In [20]:
# calculate total delay savings for passenger cars and trucks
delay_saving_df = flows_df.copy()

delay_saving_df["ea_auto_delay_savings"] = delay_saving_df["ea_auto_flow"] * (delay_saving_df["ea_delay"]/ 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["am_auto_delay_savings"] = delay_saving_df["am_auto_flow"] * (delay_saving_df["am_delay"]/ 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["md_auto_delay_savings"] = delay_saving_df["md_auto_flow"] * (delay_saving_df["md_delay"]/ 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["pm_auto_delay_savings"] = delay_saving_df["pm_auto_flow"] * (delay_saving_df["pm_delay"]/ 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["ev_auto_delay_savings"] = delay_saving_df["ev_auto_flow"] * (delay_saving_df["ev_delay"]/ 60.0) * (total_percent_delay_reduction / 100.0)

delay_saving_df["ea_truck_delay_savings"] = delay_saving_df["ea_truck_flow"] * (delay_saving_df["ea_delay"] / 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["am_truck_delay_savings"] = delay_saving_df["am_truck_flow"] * (delay_saving_df["am_delay"] / 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["md_truck_delay_savings"] = delay_saving_df["md_truck_flow"] * (delay_saving_df["md_delay"] / 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["pm_truck_delay_savings"] = delay_saving_df["pm_truck_flow"] * (delay_saving_df["pm_delay"] / 60.0) * (total_percent_delay_reduction / 100.0)
delay_saving_df["ev_truck_delay_savings"] = delay_saving_df["ev_truck_flow"] * (delay_saving_df["ev_delay"] / 60.0) * (total_percent_delay_reduction / 100.0)

delay_saving_df["daily_auto_delay_savings"] = (delay_saving_df["ea_auto_delay_savings"]
                                               + delay_saving_df["am_auto_delay_savings"]
                                               + delay_saving_df["md_auto_delay_savings"]
                                               + delay_saving_df["pm_auto_delay_savings"]
                                               + delay_saving_df["ev_auto_delay_savings"])

delay_saving_df["daily_truck_delay_savings"] = (delay_saving_df["ea_truck_delay_savings"]
                                                + delay_saving_df["am_truck_delay_savings"]
                                                + delay_saving_df["md_truck_delay_savings"]
                                                + delay_saving_df["pm_truck_delay_savings"]
                                                + delay_saving_df["ev_truck_delay_savings"])

auto_daily_delay_savings = delay_saving_df["daily_auto_delay_savings"].sum()
truck_daily_delay_savings = delay_saving_df["daily_truck_delay_savings"].sum()

In [21]:
auto_daily_delay_savings, truck_daily_delay_savings

(13.65460478268, 0.24710764629999998)

In [22]:
delay_savings_df = pd.DataFrame.from_dict(
    {
        'Variable': ["Total delays savings for passenger cars (hours)", "Total delay savings for trucks (hours)"],
        'Value': [auto_daily_delay_savings, truck_daily_delay_savings]
    }
)

In [23]:
delay_savings_df

,Variable,Value
0,Total delays savings for passenger cars (hours),13.654605
1,Total delay savings for trucks (hours),0.247108


In [24]:
# get emission factors
emission_factors_df = get_emission_factors(emission_df, scen_year)
emission_factors_df

,Year,Vehicle Type,CO2 RunEx Emission Factor (gr/mile),CO2 IdlEx Emission Factor (gr/hour),CO2 StrEx Emission Factor (gr/trip)
0,2035,Passenger Car,213.69,518.91,43.24
1,2035,Bus - All Fuel Types,NaN,2622.82,NaN
2,2035,Bus - Dsl,0.00,0.00,0.00
3,2035,Bus - Gas,1254.85,0.00,45.34
4,2035,Bus - NG,2053.53,0.00,0.00
5,2035,Bus - Propane,NaN,NaN,NaN
6,2035,HDT - All Weight Types,NaN,2016.87,NaN
7,2035,Light HDT,644.42,121.78,7.83
8,2035,Medium HDT,937.84,570.07,8.13
9,2035,Heavy HDT,1140.56,11763.95,0.07


In [25]:
# calculate ghg reductions 
ghg_reduction_df = calculate_ghg_reduction(auto_daily_delay_savings, truck_daily_delay_savings, emission_factors_df)
ghg_reduction_df

,Variable,Value
0,GHG reduction due to delay savings for passeng...,0.007794
1,GHG reduction due to delay savings for trucks ...,0.000548
2,Total GHG reduction (short tons),0.008342


In [26]:
results_df = pd.concat([delay_savings_df, ghg_reduction_df])

In [27]:
results_df

,Variable,Value
0,Total delays savings for passenger cars (hours),13.654605
1,Total delay savings for trucks (hours),0.247108
0,GHG reduction due to delay savings for passeng...,0.007794
1,GHG reduction due to delay savings for trucks ...,0.000548
2,Total GHG reduction (short tons),0.008342


In [28]:
# writing results
results_dict = {"GHG_Reduction": results_df, "Emission_Factors": emission_factors_df}
write_results(results_dict, output_results_filename, output_dir)